TO DO

- add an optional constraint to allow only one defender from any given team (risk mitigation)

In [1]:
import pandas as pd
import numpy as np
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [2]:
# function to solve the optimization problem
def optimize(df, max_price, expected_column):
    
    # PRELIMINARIES
    
    # Create the model
    model = LpProblem(name="FPL", sense=LpMaximize)    
    variables = [LpVariable(name=f'{ix}', cat='Binary') for ix in df.index]
    prices = [df.loc[ix,'now_cost']/10.0 for ix in df.index]
    # measure of player quality
    expected_points = [df.loc[ix,expected_column] for ix in df.index]
    goalkeepers = [1.0 if df.loc[ix,'element_type']==1 else 0.0 for ix in df.index]
    defenders = [1.0 if df.loc[ix,'element_type']==2 else 0.0 for ix in df.index]
    midfielders = [1.0 if df.loc[ix,'element_type']==3 else 0.0 for ix in df.index]
    forwards = [1.0 if df.loc[ix,'element_type']==4 else 0.0 for ix in df.index]

    # CONSTRAINTS

    # select 11 players
    model += lpSum(variables) == 11
    # set maximum price for starting 11
    model += np.dot(prices,variables) <= max_price
    # only 1 goalkeeper
    model += np.dot(goalkeepers,variables) == 1
    # at least 3 defenders
    model += np.dot(defenders,variables) >= 3
    # at most 5 defenders
    model += np.dot(defenders,variables) <= 5
    # at most 5 midfielders
    model += np.dot(midfielders,variables) <= 5
    # at least 1 forward
    model += np.dot(forwards,variables) >= 1
    # at most 3 forwards
    model += np.dot(forwards,variables) <= 3
    
    # OBJECTIVE
    # if possible, SHOULD ADD CAPTAIN'S DOUBLE POINTS TO THE OBJECTIVE
    model += np.dot(expected_points,variables)

    # SOLVE OPTIMIZATION
    
    status = model.solve()
    print(f'Status: {LpStatus[model.status]}')
    #print(f'Mean total points per gameweek: {model.objective.value()}')

    players = [int(str(var)) for var in model.variables() if var.value()==1]
    dream_team = df.loc[players]
    cost = (dream_team['now_cost']/10.0).sum()
    exp_points = dream_team[expected_column].sum() + dream_team[expected_column].max()
    print(f'Cost: {cost}')
    if expected_column == 'adjusted points per game':
        print(f'Expected points per week: {exp_points}')
    elif expected_column == 'xPoints_next10':
        exp_points = exp_points / 10
        print(f'Expected points per week: {exp_points}')
    else:
        print(f'Expected points: {exp_points}')
    display(dream_team[['element_type','web_name','now_cost','games played',expected_column]].sort_values('element_type'))

In [3]:
gameweek = 15
# fetch FPL data
filepath = '../data/fpl/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,...,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 25 prediction
id,,,,,,,,,,,,,,,,,,,,,
1,6.438964,2.146321,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.089483,NaN,NaN,NaN,NaN,NaN,NaN,35.142914,0,NaN
2,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.937826,0,NaN
3,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.041167,0,NaN
4,50.411000,3.600786,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,10.5,3.6,3.8,2.4,6.8,4.071,32.502103,3,NaN
5,3.738964,1.246321,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.910517,1.0,NaN,NaN,NaN,NaN,NaN,18.924435,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.875928,0,2.786111
583,40.800000,3.400000,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,1.4,2.4,7.8,2.0,6.400,34.782298,1,3.487130
611,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.386210,0,3.325335


In [4]:
# only consider players that have played at least some minimum number of games
df = df[df['games played']>=3]
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,...,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 25 prediction
id,,,,,,,,,,,,,,,,,,,,,
1,6.438964,2.146321,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.089483,NaN,NaN,NaN,NaN,NaN,NaN,35.142914,0,NaN
4,50.411000,3.600786,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,10.500000,3.600000,3.800000,2.400000,6.800000,4.071000,32.502103,3,NaN
5,3.738964,1.246321,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.910517,1.000000,NaN,NaN,NaN,NaN,NaN,18.924435,1,NaN
6,18.700000,1.870000,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,NaN,NaN,NaN,1.000000,3.600000,2.000000,20.172688,0,NaN
7,6.816235,1.363247,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,-2.000000,NaN,NaN,2.332871,NaN,NaN,NaN,17.448132,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,7.300000,1.042857,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,1.000000,NaN,1.000000,NaN,NaN,1.300000,NaN,18.084965,0,1.817551
461,36.483226,3.316657,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,5.548812,4.418731,2.382085,NaN,5.048812,NaN,NaN,31.155705,0,2.999566
470,29.331295,3.666412,1,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.861196,34.436664,1,3.631310


In [5]:
optimize(df, 85.2, f'gameweek {gameweek+1} prediction')

Status: Optimal
Cost: 84.60000000000001
Expected points: 62.137317180633545


,element_type,web_name,now_cost,games played,gameweek 16 prediction
id,,,,,
270,1,de Gea,50,15.0,4.942366
127,2,Rüdiger,61,14.0,4.576765
135,2,Chilwell,57,6.0,5.135636
237,2,Alexander-Arnold,81,13.0,5.082798
138,3,Mount,75,12.0,4.920009
210,3,Tielemans,64,11.0,4.854860
233,3,Salah,131,15.0,5.759519
240,3,Jota,80,14.0,5.381866
265,3,Foden,81,9.0,5.287307


In [6]:
optimize(df, 85.2, 'adjusted points per game')

Status: Optimal
Cost: 82.9
Expected points per week: 78.16051917784328


,element_type,web_name,now_cost,games played,adjusted points per game
id,,,,,
475,1,Sá,50,15.0,4.322270
135,2,Chilwell,57,6.0,6.054213
142,2,James,62,11.0,5.213652
237,2,Alexander-Arnold,81,13.0,6.985187
256,2,Cancelo,68,15.0,5.843690
233,3,Salah,131,15.0,8.238226
240,3,Jota,80,14.0,6.066536
250,3,Gündogan,73,10.0,6.004206
265,3,Foden,81,9.0,6.078117


In [7]:
df = df.drop([268])
optimize(df, 85.2, 'adjusted points per game')

Status: Optimal
Cost: 83.69999999999999
Expected points per week: 77.0162750624864


,element_type,web_name,now_cost,games played,adjusted points per game
id,,,,,
475,1,Sá,50,15.0,4.322270
135,2,Chilwell,57,6.0,6.054213
142,2,James,62,11.0,5.213652
237,2,Alexander-Arnold,81,13.0,6.985187
256,2,Cancelo,68,15.0,5.843690
138,3,Mount,75,12.0,5.183512
233,3,Salah,131,15.0,8.238226
240,3,Jota,80,14.0,6.066536
250,3,Gündogan,73,10.0,6.004206


In [8]:
#df = df.drop([119])
optimize(df, 85.2, 'xPoints_next10')

Status: Optimal
Cost: 83.6
Expected points per week: 58.54610340595245


,element_type,web_name,now_cost,games played,xPoints_next10
id,,,,,
270,1,de Gea,50,15.0,44.209645
237,2,Alexander-Arnold,81,13.0,47.736055
256,2,Cancelo,68,15.0,41.727653
411,2,Cresswell,54,13.0,43.790847
233,3,Salah,131,15.0,54.699742
240,3,Jota,80,14.0,51.984065
265,3,Foden,81,9.0,54.349788
420,3,Bowen,65,15.0,45.996155
423,3,Fornals,60,15.0,44.714424
